# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989




## Answers

Q1-> C: 8.382332347441762

Q2-> B: 3.605551275463989



# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [26]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [6]:
import dlt

# Define duckdb connection with dataset_name = 'persons_data'
pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons_data')

# first load people_1() generator into "persons" table
info = pipeline.run(people_1(), table_name="persons")

print(info)



Pipeline dlt_colab_kernel_launcher load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707912491.5244505 is LOADED and contains no failed jobs


In [14]:
# calculate the sum of ages of all people_1
import duckdb
#connect to duckdb with pipeline
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

#from connecting pipeline, connect to dataset (set dataset path)
conn.sql(f"SET search_path = {pipeline.dataset_name}")


select_table = conn.sql("select * from persons")
print(select_table)

# calculate the sum of ages of all people_1
sum_ages_people_1 = conn.sql("select sum(age) from persons")
print(sum_ages_people_1)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707912491.5244505 │ zlZNhVzNFXcKnw │
│     2 │ Person_2 │    27 │ City_A  │ 1707912491.5244505 │ ty1HpKplk7nctA │
│     3 │ Person_3 │    28 │ City_A  │ 1707912491.5244505 │ L6EH9qOMGfjTIw │
│     4 │ Person_4 │    29 │ City_A  │ 1707912491.5244505 │ XkVIvCRBQODthw │
│     5 │ Person_5 │    30 │ City_A  │ 1707912491.5244505 │ hNP4RwLDjNY40w │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘



In [15]:
# Second load people_2() generator into "persons" table with append mode
info = pipeline.run(people_2(), table_name="persons", write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707914068.8244052 is LOADED and contains no failed jobs


In [24]:
people2 = conn.sql("select * from persons")
print(people2)

# calculate the sum of ages of all people
sum_ages = conn.sql("select sum(age) from persons")
print(sum_ages)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707912491.5244505 │ zlZNhVzNFXcKnw │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707912491.5244505 │ ty1HpKplk7nctA │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707912491.5244505 │ L6EH9qOMGfjTIw │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707912491.5244505 │ XkVIvCRBQODthw │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707912491.5244505 │ hNP4RwLDjNY40w │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707914068.8244052 │ lMzIHE3eC7BuvA │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707914068.8244052 │ hb/h6OBSX4rI7A │ Job_4      │
│     5 │ 

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [18]:
info = pipeline.run(people_1, table_name="persons_merge")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707914805.0025027 is LOADED and contains no failed jobs


In [19]:
select_people1 = conn.sql("select * from persons_merge")
print(select_people1)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707914805.0025027 │ /ASQFpaf/SQ8JA │
│     2 │ Person_2 │    27 │ City_A  │ 1707914805.0025027 │ oo1tOV4X4I6pvw │
│     3 │ Person_3 │    28 │ City_A  │ 1707914805.0025027 │ l8Q/vOigqyvHqw │
│     4 │ Person_4 │    29 │ City_A  │ 1707914805.0025027 │ 2shhQdHAj+tA6w │
│     5 │ Person_5 │    30 │ City_A  │ 1707914805.0025027 │ UKDH7ClwnmWong │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘



In [27]:
info = pipeline.run(people_2, table_name="persons_merge", write_disposition="merge", primary_key="id")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.67 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707915226.4782197 is LOADED and contains no failed jobs


In [29]:
after_merge = conn.sql("select * from persons_merge")
print(after_merge)

after_merge_rows_count = conn.sql("select count(*) from persons_merge")
print(after_merge_rows_count)

after_merge_sum_age = conn.sql("select sum(age) from persons_merge")
print(after_merge_sum_age)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707914805.0025027 │ /ASQFpaf/SQ8JA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707914805.0025027 │ oo1tOV4X4I6pvw │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1707915226.4782197 │ P3Ip0bP4DZ9U5w │ Job_5      │
│     8 │ Person_8 │    38 │ City_B  │ 1707915226.4782197 │ ZAl4YokHpjSPSA │ Job_8      │
│     7 │ Person_7 │    37 │ City_B  │ 1707915226.4782197 │ 7SierPrrbtAiiQ │ Job_7      │
│     4 │ Person_4 │    34 │ City_B  │ 1707915226.4782197 │ ymwiRo6Qz0YPMw │ Job_4      │
│     3 │ Person_3 │    33 │ City_B  │ 1707915226.4782197 │ p6AtYzswHl7p0A │ Job_3      │
│     6 │ 